In [141]:
# !pip install beautifulsoup4
# !pip install bs4
# !pip install geopy

In [1]:
# For query a website
import urllib.request
import random
import time

In [2]:
# To parse a html page and extract required data
from bs4 import BeautifulSoup
import numpy as np
import re


In [3]:
def findOrNan(obj,type):
    if type=='text':
        return obj.text if hasattr(obj,'text') else 'null'
    elif type == 'content':
        return obj['content'] if hasattr(obj,'content') else 'null'
    elif type== 'datetime':
        return obj['datetime'] if hasattr(obj,'datetime') else 'null'

In [4]:
import pickle

# Initialize variables
start_from = 0  # Start from the beginning by default

# Check if there is a saved state
try:
    with open('loop_state.pkl', 'rb') as file:
        start_from = pickle.load(file)
except FileNotFoundError:
    pass

In [5]:

entries=[]
iter=0

for i in range(437,500):
   print("Page Number:",i)
   url = f"https://www.kijiji.ca/b-apartments-condos/ontario/page-{i}/c37l9004"

   webPage = urllib.request.urlopen(url)
   parsedPage = BeautifulSoup(webPage)

   item_list = parsedPage.find_all('ul', attrs={'data-testid': 'srp-search-list'})
   for list in item_list:
      # Finding each <a></a> tag and extracting id="listing-link" for getting more details of the product
      a_tags_with_link_id = list.find_all('a', attrs={"data-testid":"listing-link"})
      for tag in a_tags_with_link_id:

         iter=iter+1
         # print("https://www.kijiji.ca"+tag['href'])
         page_details = {}
         # ad_id=""


         listing = urllib.request.urlopen("https://www.kijiji.ca"+tag['href'])
         # listing = urllib.request.urlopen("https://www.kijiji.ca/v-apartments-condos/city-of-toronto/annex-1-bedroom-with-private-backyard/1685925449")

         
         page_details_html= BeautifulSoup(listing, 'html.parser')

         # id = page_details_html.find('li', class_='currentCrumb-2810804557')
         # if id:
         #    ad_id = findOrNan(id.find('a', class_='adId-1017171818'),'text')
         #    print("ad_id",ad_id)

         # Extracting usefull details from HTML page
         title = findOrNan(page_details_html.find('h1', class_='title-4206718449'),'text')
         page_details['title'] = title

         price = findOrNan(page_details_html.find('span', content=True),'content')
         page_details['price'] = price

         price_condition =findOrNan(page_details_html.find('span', class_='utilities-3805585157'),'text')
         page_details['price_condition'] = price_condition

         address = findOrNan(page_details_html.find('span', itemprop='address'),'text') 
         page_details['address'] = address

         posted_date = findOrNan(page_details_html.find('time'),'datetime')
         page_details['posted_date'] = posted_date


         quick_view = page_details_html.find('div', class_='titleAttributes-183069789')
         if quick_view:
            for quick_view_detail in quick_view.find_all('li'):
               quick_view_detail_value = findOrNan(quick_view_detail.find('span', class_='noLabelValue-774086477'),'text')
               if('Bedrooms' in quick_view_detail_value):
                  page_details['Bedrooms'] = quick_view_detail_value
               elif 'Bathrooms' in quick_view_detail_value:
                  page_details['Bathrooms'] = quick_view_detail_value
               else:
                  page_details['unit_type'] = quick_view_detail_value


         utilities_included = page_details_html.find('h4', string='Utilities Included')
         pattern = r'(Yes|No): (.*)'
         if utilities_included:
            utilities_list = utilities_included.find_next('ul').find_all('li')
            for utility in utilities_list:
               svg_tag = utility.find('svg')
               if svg_tag:
                  aria_label = svg_tag.get('aria-label')
                  match = re.match(pattern, aria_label)
                  if match:
                     value = match.group(1)
                     label = match.group(2)
                     page_details[label] = value


         wifi_and_more = page_details_html.find('h4', string='Wi-Fi and More')
         if wifi_and_more:
            parent_div = wifi_and_more.find_parent('div')

            features_list = parent_div.find_next('ul').find_all('li')
            for feature in features_list:
               page_details[feature.text] = True

         parking = page_details_html.find('dt', string='Parking Included')
         if parking:
            features_list = parking.find_next('dd')
            page_details['parking'] = findOrNan(features_list,'text')

         agreement = page_details_html.find('dt', string='Agreement Type')
         if agreement:
            agreement_type = agreement.find_next('dd')
            page_details['agreement_type'] = findOrNan(agreement_type,'text')

         moveIn = page_details_html.find('dt', string='Move-In Date')
         if moveIn:
            moveIn_date = moveIn.find_next('dd')
            page_details['moveIn_date'] = findOrNan(moveIn_date,'text')

         pet = page_details_html.find('dt', string='Pet Friendly')
         if pet:
            pet_friendly = pet.find_next('dd')
            page_details['pet_friendly'] = findOrNan(pet_friendly,'text')

         size = page_details_html.find('dt', string='Size (sqft)')
         if size:
            size_sqrt = size.find_next('dd')
            page_details['size_sqrt'] = findOrNan(size_sqrt,'text')

         furnished = page_details_html.find('dt', string='Furnished')
         if furnished:
            furnished_info = furnished.find_next('dd')
            page_details['furnished_info'] = findOrNan(furnished_info,'text')

         appliances = page_details_html.find('h4', string='Appliances')
         if appliances:
            parent_div = appliances.find_parent('div')

            appliances_list = parent_div.find_next('ul').find_all('li')
            for feat in appliances_list:
               page_details[feat.text] = True

         air_conditioning = page_details_html.find('dt', string='Air Conditioning')
         if air_conditioning:
            air_conditioning_info = air_conditioning.find_next('dd')
            page_details['air_conditioning_info'] = findOrNan(air_conditioning_info,'text')

         personal_outdoor_space = page_details_html.find('dt', string='Personal Outdoor Space')
         if personal_outdoor_space:
            personal_outdoor_space_info = personal_outdoor_space.find_next('dd')
            page_details['personal_outdoor_space_info'] = findOrNan(personal_outdoor_space_info,'text')

         smoking = page_details_html.find('dt', string='Smoking Permitted')
         if smoking:
            smoking_info = smoking.find_next('dd')
            page_details['smoking_info'] = findOrNan(smoking_info,'text')

         amenities = page_details_html.find('h4', string='Amenities')
         if amenities:
            parent_div = amenities.find_parent('div')

            amenities_list = parent_div.find_next('ul').find_all('li')
            for feat in amenities_list:
               page_details[feat.text] = True



         description_section = page_details_html.find('h3', string='Description')
         if description_section:
            parent_div = description_section.find_parent('div')
            description_text = parent_div.get_text(separator='\n')
            page_details['description']=description_text


         # entries.append(page_details)
         # print("-------------------------",page_details)
         # print(">>>>",entries)
         with open('vals3.txt', 'a') as file:
            file.write(str(page_details) + '\n')

         if iter % 10 == 0:  # Save every 100 iterations
            with open('loop_state.pkl', 'wb') as file:
                  pickle.dump(i, file)

         delay = random.randint(10, 30)
         time.sleep(delay)



Page Number: 437
Page Number: 438
Page Number: 439
Page Number: 440
Page Number: 441
Page Number: 442
Page Number: 443
Page Number: 444
Page Number: 445
Page Number: 446
Page Number: 447
Page Number: 448
Page Number: 449
Page Number: 450
Page Number: 451
Page Number: 452
Page Number: 453
Page Number: 454
Page Number: 455
Page Number: 456
Page Number: 457
Page Number: 458


URLError: <urlopen error [WinError 10054] An existing connection was forcibly closed by the remote host>

In [148]:
import pandas as pd
# Initialize lists to store data
price = []
title = []
price_condition=[]

# Read the content of the text file and extract data
with open('vals.txt', 'r') as file:
    for line in file:
        data = eval(line)  # Convert string to dictionary
        price.append(data['price'])
        title.append(data['title'])
        price_condition.append(data['price_condition'])


# Create a pandas DataFrame
df = pd.DataFrame({'price': price, 'title': title,'price_condition':price_condition})

df

,price,title,price_condition
0,2395.,2 BEDROOM - BRAND NEW NEVER LIVED IN UNIT,Some Utilities Included
1,2150.,3 Bedroom Apartment for Rent in Georgetown!,Some Utilities Included
2,1625.,Avalon Park: Apartment for rent in Southeast O...,Some Utilities Included
